# Loss Comparison between Numpy and Tensorflow

This document aims to compare the losses that are used in SCORE and ShapeDeconv and ultimately harmonize them.

## Load present losses

<b>Note:</b> Only the differentiable part of the loss is considered is the case of SCORE.

Setting required constants and variables:

In [48]:
# import libraries
import numpy as np
import cadmos_lib as cl
import tensorflow as tf
import sys
from numpy.linalg import norm

# declare constants that are common for single and multi window constraint
n_row = n_col = 96
gamma = 1
U = cl.makeUi(n_row,n_col)

# generate mock data
y_pred,win = np.random.rand(2,n_row,n_col)
y_true = np.zeros((n_row,n_col))
residual = y_true-y_pred

#declare constants that are specific to single window constraint
muw = np.array([y_pred.size/norm(win*Ui)**2/U.shape[0] for Ui in U])

#declare constants that are specific to single window constraint
n_shearlet = 3
shearlets,shearlets_adj = cl.get_shearlets(n_row,n_col,n_shearlet)
psu = np.array([cl.convolve_stack(ui,shearlets_adj) for ui in U]) #shealret adjoint of U, i.e Psi^{Star}(U)
mus = cl.comp_mu(psu)

### 1. Single Window SCORE Loss

In [50]:
def loss_SW_score(residual):
    data_fid = norm(residual)**2/2.
    print("data_fid=",data_fid)
    shape_constraint = gamma*(np.array([m*((residual*win*u).sum())**2
                                        for m,u in zip(muw,U)])/2.).sum()
    print("shape_constraint=",shape_constraint)
    return data_fid+shape_constraint

print("loss=",loss_SW_score(residual))

data_fid= 1518.304569214998
shape_constraint= 4861589.5406159
loss= 4863107.845185115


### 2. Single Window ShapeDecon Loss

In [51]:
def loss_SW_unet(y_pred,y_true):
    y_pred = tf.reshape(y_pred, [1,*y_pred.shape,1])
    y_true = tf.reshape(y_true, [1,*y_true.shape,1])
    residual = y_true-y_pred
    data_fid = tf.keras.backend.mean(tf.keras.backend.square(y_true-y_pred), axis=-1)
    print("data_fid=",tf.keras.backend.get_value(data_fid))
    window = tf.reshape(win, [1,*win.shape,1])
    mu = tf.reshape(muw, [1,*muw.shape,1,1])
    shape_constraint=0
    for i in range(6):
        shape_constraint+=gamma*mu[:,i,0,0]*\
        (tf.keras.backend.square(
            (tf.keras.backend.sum((residual)*window*U[i],axis=(1,2,3)))))/2
    shape_constraint/=(96*96)
    shape_constraint=tf.keras.backend.expand_dims((tf.keras.backend.expand_dims(shape_constraint, axis=-1)),axis=-1)
    print("shape_constraint=",tf.keras.backend.get_value(shape_constraint))
    return data_fid+shape_constraint

print("loss=",tf.keras.backend.get_value(loss_SW_unet(y_pred,y_true)))

data_fid= [[[5.71142011e-01 8.55915273e-01 7.27389555e-02 ... 1.53672920e-01
   1.20383979e-03 4.78931767e-01]
  [3.81475788e-01 6.95156378e-03 1.79247123e-02 ... 1.46474129e-01
   8.57399244e-02 1.77509751e-02]
  [5.06797008e-02 2.93113471e-01 2.09339225e-01 ... 6.45609802e-01
   2.02292828e-03 7.87914969e-01]
  ...
  [3.24279222e-01 9.99242840e-01 8.12453649e-01 ... 9.72553561e-01
   1.08251992e-01 2.93266674e-01]
  [5.31358150e-01 8.78613539e-04 2.39181084e-01 ... 5.91011609e-01
   4.49321037e-01 4.01649819e-02]
  [5.30482238e-01 6.62608107e-01 4.17743277e-01 ... 6.19546017e-01
   2.05129163e-02 8.88865229e-03]]]
shape_constraint= [[[4876707.37728474]]]
loss= [[[4876707.94842675 4876708.23320001 4876707.45002369 ...
   4876707.53095766 4876707.37848858 4876707.85621651]
  [4876707.75876053 4876707.3842363  4876707.39520945 ...
   4876707.52375887 4876707.46302466 4876707.39503571]
  [4876707.42796444 4876707.67039821 4876707.58662396 ...
   4876708.02289454 4876707.37930767 4876708.

### 3. Rewriting Single Window ShapeDecon Loss

In [52]:
def loss_SW_keras(y_pred,y_true):
    y_pred = tf.reshape(y_pred, [1,*y_pred.shape,1])
    y_true = tf.reshape(y_true, [1,*y_true.shape,1])
    residual = y_true-y_pred
    data_fid = tf.keras.backend.sum(tf.keras.backend.square(y_true-y_pred),axis=(1,2,3))/2.
    print("data_fid=",tf.keras.backend.get_value(data_fid))
    window = tf.reshape(win, [1,*win.shape,1])
    U_tensor = tf.reshape(U, [U.shape[0],1,*U.shape[1:],1])
    mu = tf.reshape(muw, [*muw.shape])
    shape_constraint=0
    for i in range(6):
        shape_constraint+=gamma*mu[i]*\
        tf.keras.backend.square(
            (tf.keras.backend.sum(residual*window*U_tensor[i],axis=(1,2,3))))/2
    print("shape_constraint=",tf.keras.backend.get_value(shape_constraint))
    return data_fid+shape_constraint

print("loss=",tf.keras.backend.get_value(loss_SW_keras(y_pred,y_true)))

data_fid= [1518.30456921]
shape_constraint= [4861589.5406159]
loss= [4863107.84518512]


### 4. Multi Window SCORE Loss

In [5]:
def loss_MW_score(residual):
    data_fid = np.linalg.norm(residual)**2/2.
    print("data_fid=",data_fid)
    shape_constraint = gamma*(np.array(\
            [[mu_ij*((residual*psu_ij).sum())**2\
              for mu_ij,psu_ij in zip(mu_j, psu_j)]\
            for mu_j,psu_j in zip(mus,psu)])/2.).sum()
    print("shape_constraint=",shape_constraint)
    loss = data_fid+shape_constraint
    return loss

print("loss=",loss_MW_score(residual))

data_fid= 1532.8932077213979
shape_constraint= 2646.1208671422955
loss= 4179.014074863693


### 5. Multi Window ShapeDecon Loss

In [6]:
def loss_MW_unet(y_pred,y_true):
    y_pred = tf.reshape(y_pred, [1,*y_pred.shape,1])
    y_true = tf.reshape(y_true, [1,*y_true.shape,1])
    mu = tf.reshape(mus, [*mus.shape])
    residual=y_pred-y_true
    data_fid = tf.keras.backend.mean(tf.keras.backend.square(y_true-y_pred), axis=-1)
    print("data_fid=",tf.keras.backend.get_value(data_fid))
    shape_constraint=0
    for i in range(6):
        for j in range(27):
            shape_constraint+=mu[i,j]*\
            tf.keras.backend.square(
                tf.keras.backend.sum(residual*psu[i,j],axis=(1,2,3)))
    shape_constraint=shape_constraint*gamma/(n_row*n_col)
    shape_constraint=tf.keras.backend.expand_dims((tf.keras.backend.expand_dims(shape_constraint, axis=-1)),axis=-1)
    print("shape_constraint=",tf.keras.backend.get_value(shape_constraint))
    loss=data_fid+shape_constraint
    return loss

print("loss=",tf.keras.backend.get_value(loss_MW_unet(y_pred,y_true)))

data_fid= [[[0.62095859 0.1543961  0.1188128  ... 0.72973828 0.75477366 0.67996461]
  [0.62062014 0.50929613 0.15668238 ... 0.05794295 0.63565033 0.01797836]
  [0.64558407 0.16339648 0.38045125 ... 0.0119064  0.47703046 0.47996539]
  ...
  [0.07559364 0.95912179 0.78733917 ... 0.09940566 0.04430517 0.5981372 ]
  [0.39941973 0.39316254 0.63489125 ... 0.11621672 0.97232445 0.00110615]
  [0.2089486  0.30982124 0.48215901 ... 0.36285843 0.01370121 0.03027108]]]
shape_constraint= [[[5311.56130085]]]
loss= [[[5312.18225943 5311.71569694 5311.68011365 ... 5312.29103913
   5312.3160745  5312.24126545]
  [5312.18192099 5312.07059698 5311.71798323 ... 5311.6192438
   5312.19695117 5311.5792792 ]
  [5312.20688491 5311.72469732 5311.94175209 ... 5311.57320724
   5312.0383313  5312.04126623]
  ...
  [5311.63689449 5312.52042263 5312.34864001 ... 5311.66070651
   5311.60560602 5312.15943805]
  [5311.96072058 5311.95446338 5312.1961921  ... 5311.67751757
   5312.53362529 5311.562407  ]
  [5311.770249

### 6. Rewriting Multi Window ShapeDecon Loss

In [7]:
def loss_MW_keras(y_pred,y_true):
    y_pred = tf.reshape(y_pred, [1,*y_pred.shape,1])
    y_true = tf.reshape(y_true, [1,*y_true.shape,1])
    mu = tf.reshape(mus, [*mus.shape])
    psu_tensor = tf.reshape(psu, [*psu.shape[:2],1,*psu.shape[2:],1])
    residual=y_pred-y_true
    data_fid = tf.keras.backend.sum(tf.keras.backend.square(y_true-y_pred),axis=(1,2,3))/2.
    print("data_fid=",tf.keras.backend.get_value(data_fid))
    shape_constraint=0
    for i in range(6):
        for j in range(27):
            shape_constraint+=gamma*mu[i,j]*\
            tf.keras.backend.square(
                tf.keras.backend.sum(residual*psu_tensor[i,j],axis=(1,2,3)))/2.
    print("shape_constraint=",tf.keras.backend.get_value(shape_constraint))
    loss=data_fid+shape_constraint
    return loss

print("loss=",tf.keras.backend.get_value(loss_MW_keras(y_pred,y_true)))

data_fid= [1532.89320772]
shape_constraint= [2646.12086714]
loss= [4179.01407486]
